In [4]:
import os
import json
import pathlib
import time

import requests
from requests.auth import HTTPBasicAuth
from helper import planet_auth

In [5]:
# Setup the API Key from the `PL_API_KEY` environment variable
PLANET_API_KEY = planet_auth()

In [6]:
# Setup Planet Data API base URL
URL = "https://api.planet.com/data/v1"

# Setup the session
session = requests.Session()

# Authenticate
session.auth = (PLANET_API_KEY, "")

In [7]:
# Helper function to printformatted JSON using the json module
def p(data):
    print(json.dumps(data, indent=2))

In [8]:
# Make a GET request to the Planet Data API
res = session.get(URL)

In [20]:
# Response status code
res.status_code

200

In [151]:
## long geometry filter following (collapse/uncollapse by double clicking to the left of the cell)

In [26]:
geom =  {
  "type": "Polygon",
  "coordinates":  [
          [
            [
              -16.337415533922837,
              16.29831977936631
            ],
            [
              -16.337415533922837,
              16.27268392173528
            ],
            [
              -16.28457599804048,
              16.27268392173528
            ],
            [
              -16.28457599804048,
              16.29831977936631
            ],
            [
              -16.337415533922837,
              16.29831977936631
            ]
          ]
        ]
      }

# Setup Geometry Filter
geometry_filter = {
    "type": "GeometryFilter",
    "field_name": "geometry",
    "config": geom
}

In [27]:
# Create filter object for all imagery captured during the 2nd - 7th of May 2018
date_filter = {
    "type": "DateRangeFilter", # Type of filter -> Date Range
    "field_name": "acquired", # The field to filter on: "acquired" -> Date on which the "image was taken"
    "config": {
        "gte": "2019-05-01T00:00:00.000Z",
        "lte": "2019-05-30T00:00:00.000Z"
    }
}

In [31]:
# only get images which have <50% cloud coverage
cloud_cover_filter = {
  "type": "RangeFilter",
  "field_name": "cloud_cover",
  "config": {
    "lte": 0.5
  }
}


In [34]:
# Setup an "AND" logical filter
combined_filter = {
    "type": "AndFilter",
    "config": [geometry_filter, date_filter,cloud_cover_filter]
}

# Print the logical filter
#p(and_filter)

In [35]:
# this large search filter produces all PlanetScope imagery for the 1st -30th of May, 2019 in the lower basin of the Senegal River

item_type = "PSScene"

# API request object
search_request = {
  "item_types": [item_type], 
  "filter": combined_filter
}

# Anida_2019May_1to30 = {
#   "name": "Anida_2019May_1to30",
#   "item_types": "PSScene",
#   "product_bundle": "analytic_udm2",
#   "filter": and_filter
# }

In [36]:
# fire off the POST request
search_result = \
  requests.post(
    'https://api.planet.com/data/v1/quick-search',
    auth=HTTPBasicAuth(PLANET_API_KEY, ''),
    json=search_request)

geojson = search_result.json()

# let's look at the first result
print(list(geojson.items())[1][1][0])

{'_links': {'_self': 'https://api.planet.com/data/v1/item-types/PSScene/items/20190529_111516_101f', 'assets': 'https://api.planet.com/data/v1/item-types/PSScene/items/20190529_111516_101f/assets/', 'thumbnail': 'https://tiles.planet.com/data/v1/item-types/PSScene/items/20190529_111516_101f/thumb'}, '_permissions': ['assets.basic_analytic_4b:download', 'assets.basic_analytic_4b_rpc:download', 'assets.basic_analytic_4b_xml:download', 'assets.basic_udm2:download', 'assets.ortho_analytic_3b:download', 'assets.ortho_analytic_3b_xml:download', 'assets.ortho_analytic_4b:download', 'assets.ortho_analytic_4b_sr:download', 'assets.ortho_analytic_4b_xml:download', 'assets.ortho_udm2:download', 'assets.ortho_visual:download'], 'assets': ['basic_analytic_4b', 'basic_analytic_4b_rpc', 'basic_analytic_4b_xml', 'basic_udm2', 'ortho_analytic_3b', 'ortho_analytic_3b_xml', 'ortho_analytic_4b', 'ortho_analytic_4b_sr', 'ortho_analytic_4b_xml', 'ortho_udm2', 'ortho_visual'], 'geometry': {'coordinates': [[[

In [38]:
# extract image IDs only
image_ids = [feature['id'] for feature in geojson['features']]
print(image_ids)

31

# Place Order


In [48]:
orders_url = 'https://api.planet.com/compute/ops/orders/v2'

In [49]:

# set content type to json
headers = {'content-type': 'application/json'}


In [50]:

request = {  
   "name":"simple order",
   "products":[
      {  
         "item_ids":image_ids,
         "item_type":"PSScene",
         "product_bundle":"analytic_udm2"
      }
   ],
}


In [51]:
def place_order(request, auth):
    response = requests.post(orders_url, data=json.dumps(request), auth=auth, headers=headers)
    print(response)
    order_id = response.json()['id']
    print(order_id)
    order_url = orders_url + '/' + order_id
    return order_url


In [52]:
order_url = place_order(request, session.auth)

<Response [202]>
bef1b41a-1af2-4710-a5f0-5e3da1568222


In [53]:
def poll_for_success(order_url, auth, num_loops=30):
    count = 0
    while(count < num_loops):
        count += 1
        r = requests.get(order_url, auth=session.auth)
        response = r.json()
        state = response['state']
        print(state)
        end_states = ['success', 'failed', 'partial']
        if state in end_states:
            break
        time.sleep(10)
        
poll_for_success(order_url, session.auth)

running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running


In [54]:
r = requests.get(order_url, auth=session.auth)
response = r.json()
results = response['_links']['results']

In [55]:
[r['name'] for r in results]


['bef1b41a-1af2-4710-a5f0-5e3da1568222/PSScene/20190504_104716_1054_3B_udm2.tif',
 'bef1b41a-1af2-4710-a5f0-5e3da1568222/PSScene/20190504_104716_1054_3B_AnalyticMS_metadata.xml',
 'bef1b41a-1af2-4710-a5f0-5e3da1568222/PSScene/20190504_104716_1054_3B_AnalyticMS.tif',
 'bef1b41a-1af2-4710-a5f0-5e3da1568222/PSScene/20190504_104716_1054_metadata.json',
 'bef1b41a-1af2-4710-a5f0-5e3da1568222/PSScene/20190504_104717_1054_3B_udm2.tif',
 'bef1b41a-1af2-4710-a5f0-5e3da1568222/PSScene/20190504_104717_1054_3B_AnalyticMS_metadata.xml',
 'bef1b41a-1af2-4710-a5f0-5e3da1568222/PSScene/20190504_104717_1054_3B_AnalyticMS.tif',
 'bef1b41a-1af2-4710-a5f0-5e3da1568222/PSScene/20190504_104717_1054_metadata.json',
 'bef1b41a-1af2-4710-a5f0-5e3da1568222/PSScene/20190525_111258_103d_3B_udm2.tif',
 'bef1b41a-1af2-4710-a5f0-5e3da1568222/PSScene/20190525_111258_103d_3B_AnalyticMS_metadata.xml',
 'bef1b41a-1af2-4710-a5f0-5e3da1568222/PSScene/20190525_111258_103d_3B_AnalyticMS.tif',
 'bef1b41a-1af2-4710-a5f0-5e3da

In [56]:
def download_results(results, overwrite=False):
    results_urls = [r['location'] for r in results]
    results_names = [r['name'] for r in results]
    print('{} items to download'.format(len(results_urls)))
    
    for url, name in zip(results_urls, results_names):
        path = pathlib.Path(os.path.join('data', name))
        
        if overwrite or not path.exists():
            print('downloading {} to {}'.format(name, path))
            r = requests.get(url, allow_redirects=True)
            path.parent.mkdir(parents=True, exist_ok=True)
            open(path, 'wb').write(r.content)
        else:
            print('{} already exists, skipping {}'.format(path, name))

In [57]:
zip_delivery = {"delivery": {"single_archive": True, "archive_type": "zip"}}
request_zip = request.copy()
request_zip.update(zip_delivery)
request_zip

{'name': 'simple order',
 'products': [{'item_ids': ['20190529_111516_101f',
    '20190525_111258_103d',
    '20190525_111257_103d',
    '20190524_111515_1035',
    '20190522_111431_1027',
    '20190522_111432_1027',
    '20190522_111337_1011',
    '20190521_104632_0f2a',
    '20190521_104631_0f2a',
    '20190521_111231_1003',
    '20190521_111230_1003',
    '20190520_111253_1035',
    '20190520_111252_1035',
    '20190519_111410_103a',
    '20190518_111204_1027',
    '20190516_111547_1008',
    '20190514_111510_1034',
    '20190513_111341_103e',
    '20190511_111206_101b',
    '20190511_111207_101b',
    '20190510_111239_1034',
    '20190508_112448_39_1058',
    '20190508_112450_46_1058',
    '20190507_111335_103b',
    '20190507_111334_103b',
    '20190504_104717_1054',
    '20190504_104716_1054',
    '20190504_113252_06_105c',
    '20190503_111512_0f28',
    '20190502_111217_1040',
    '20190502_111216_1040'],
   'item_type': 'PSScene',
   'product_bundle': 'analytic_udm2'}],
 'deli

In [58]:
order_url = place_order(request_zip, auth=session.auth)


<Response [202]>
169df078-8ddf-46fc-8147-81fe8e9b2371


In [63]:
poll_for_success(order_url, session.auth)


running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
success


In [64]:
r = requests.get(order_url, auth=session.auth)
response = r.json()
results = response['_links']['results']

In [61]:
response

{'_links': {'_self': 'https://api.planet.com/compute/ops/orders/v2/169df078-8ddf-46fc-8147-81fe8e9b2371'},
 'created_on': '2024-07-06T21:06:55.384Z',
 'delivery': {'archive_filename': 'output.zip',
  'archive_type': 'zip',
  'single_archive': True},
 'error_hints': [],
 'id': '169df078-8ddf-46fc-8147-81fe8e9b2371',
 'last_message': 'Waiting for Zip to complete',
 'last_modified': '2024-07-06T21:12:22.831Z',
 'name': 'simple order',
 'products': [{'item_ids': ['20190529_111516_101f',
    '20190525_111258_103d',
    '20190525_111257_103d',
    '20190524_111515_1035',
    '20190522_111431_1027',
    '20190522_111432_1027',
    '20190522_111337_1011',
    '20190521_104632_0f2a',
    '20190521_104631_0f2a',
    '20190521_111231_1003',
    '20190521_111230_1003',
    '20190520_111253_1035',
    '20190520_111252_1035',
    '20190519_111410_103a',
    '20190518_111204_1027',
    '20190516_111547_1008',
    '20190514_111510_1034',
    '20190513_111341_103e',
    '20190511_111206_101b',
    '201

In [65]:
download_results(results)


2 items to download
downloading 169df078-8ddf-46fc-8147-81fe8e9b2371/output.zip to data/169df078-8ddf-46fc-8147-81fe8e9b2371/output.zip
downloading 169df078-8ddf-46fc-8147-81fe8e9b2371/manifest.json to data/169df078-8ddf-46fc-8147-81fe8e9b2371/manifest.json
